In [1]:
import json
import re

# Path to your JSON file
file_path = '../Scraper/file_of_data.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
    try:
        # Load JSON data into a Python dictionary or list
        data = json.load(file)
        # Example: Accessing specific data (assuming JSON is a dictionary)
        # print(data['key'])
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


| **Strike Rate / Runs** | **0-10**       | **10-20**        | **20-50**        | **More than 50** |
|------------------------|----------------|------------------|------------------|------------------|
| **0-50**               | Don't mention  | Don't mention    | Don't mention    | Good             |
| **50-80**              | Don't mention  | Don't mention    | Slow start       | Good start       |
| **80-120**             | Don't mention  | Don't mention    | Steady start     | Solid performance|
| **More than 120**      | Don't mention  | Try to hit       | Quick Knock      | Explosive start  |


In [2]:
def evaluate_batter_performance(runs, strike_rate):
    runs = int(runs)
    strike_rate = float(strike_rate)
    # Evaluating based on the provided table
    if runs < 10:
        return "Don't mention"
    
    if 10 <= runs < 20:
        if strike_rate < 50:
            return "Don't mention"
        elif 50 <= strike_rate < 80:
            return "Don't mention"
        elif 80 <= strike_rate < 120:
            return "Under Pressure"
        else:  # strike_rate >= 120
            return "Try to hit"
    
    if 20 <= runs < 50:
        if strike_rate < 50:
            return "Don't mention"
        elif 50 <= strike_rate < 80:
            return "Slow start"
        elif 80 <= strike_rate < 120:
            return "Controlled Aggression"
        else:  # strike_rate >= 120
            return "Blistering Start"
    
    if runs >= 50:
        if strike_rate < 50:
            return "Good"
        elif 50 <= strike_rate < 80:
            return "Anchor Role"
        elif 80 <= strike_rate < 120:
            return "Solid Performance"
        else:  # strike_rate >= 120
            return "Explosive start"

    # Fallback in case none of the conditions are met
    return "Unknown performance"


In [3]:
def calculate_run_rate(overs, runs):
  overs = float(overs)
  balls = int((overs - int(overs)) * 10)
  total_overs = int(overs) + (balls / 6)
  return int(runs) / total_overs

print(calculate_run_rate('44.5', '299'))

6.669144981412639


In [4]:
def batting_run_rate_adjective(run_rate):
    if run_rate < 4:
        return "slow and cautious"
    elif 4 <= run_rate < 6:
        return "steady"
    elif 6 <= run_rate < 8:
        return "aggressive"
    else:
        return "explosive"

def batting_chasing_adjective(chasing_run_rate, required_run_rate):
    if chasing_run_rate > required_run_rate + 2:
        return "extremely aggressive"
    elif chasing_run_rate > required_run_rate:
        return "positive and proactive"
    elif chasing_run_rate == required_run_rate:
        return "right on target"
    else:
        return "struggling to keep pace"

def generate_batting_match_summary(data):
    first_batting_team = data['First Innings']['batting Team']
    second_batting_team = data['Second Innings']['batting Team']

    # First innings stats
    first_innings_runs = data['First Innings']['batters'][-1]['Total Runs']
    first_innings_overs = float(data['First Innings']['batters'][-1]['Overs'])
    first_innings_wickets = data['First Innings']['batters'][-1]['Wickets']
    first_innings_run_rate = calculate_run_rate(first_innings_overs, first_innings_runs)

    # Create the initial statement for the first innings
    statement = (f"{first_batting_team} batted first and displayed {batting_run_rate_adjective(first_innings_run_rate)} batting, "
                 f"scoring {first_innings_runs} runs for the loss of {first_innings_wickets} wickets "
                 f"in {first_innings_overs} overs with overall runrate of {first_innings_run_rate}.")
    
    # Check if there's a second innings
    if len(data['Second Innings']['batters']) != 0:
        # Second innings stats
        second_innings_runs = data['Second Innings']['batters'][-1]['Total Runs']
        second_innings_overs = float(data['Second Innings']['batters'][-1]['Overs'])
        second_innings_wickets = data['Second Innings']['batters'][-1]['Wickets']
        second_innings_run_rate = calculate_run_rate(second_innings_overs, second_innings_runs)
        
        # Calculate the required run rate for the chase
        required_run_rate = first_innings_run_rate

        # Get the adjective for the chasing team's performance
        chase_description = batting_chasing_adjective(second_innings_run_rate, required_run_rate)
        
        # Add the second innings summary
        statement += (f" In response, {second_batting_team} came to chase the target of {first_innings_runs} runs. "
                      f"Their batting was {chase_description}, scoring {second_innings_runs} runs in {second_innings_overs} overs, "
                      f"and losing {second_innings_wickets} wickets with overall runrate of {second_innings_run_rate}.")
    
    # Add the match conclusion (no win/loss assumption)
    statement += f" The match concluded with {data['match Info']['match result']}."
    
    return statement

In [5]:
def run_rate_adjective(run_rate):
    if run_rate < 4:
        return "remarkable"
    elif 4 <= run_rate < 6:
        return "disciplined"
    elif 6 <= run_rate < 8:
        return "costly"
    else:
        return "expensive"

def chasing_adjective(chasing_run_rate, required_run_rate):
    if chasing_run_rate > required_run_rate + 2:
        return "aggressive"
    elif chasing_run_rate > required_run_rate:
        return "competitive"
    elif chasing_run_rate == required_run_rate:
        return "on par"
    else:
        return "struggling"
        
def generate_bowling_match_summary(data):
    first_batting_team = data['First Innings']['batting Team']
    second_batting_team = data['Second Innings']['batting Team']
    
    # First innings stats
    first_innings_runs = data['First Innings']['batters'][-1]['Total Runs']
    first_innings_overs = float(data['First Innings']['batters'][-1]['Overs'])
    first_innings_wickets = data['First Innings']['batters'][-1]['Wickets']
    first_innings_run_rate = calculate_run_rate(first_innings_overs, first_innings_runs)
    # Create the initial statement for the first innings
    statement = (f"{second_batting_team} bowled first and displayed {run_rate_adjective(first_innings_run_rate)} bowling, "
                 f"restricting {first_batting_team} to {first_innings_runs} runs and taking {first_innings_wickets} wickets "
                 f"in {first_innings_overs} overs.")
    
    # Check if there's a second innings
    if len(data['Second Innings']['batters']) != 0:
        # Second innings stats
        second_innings_runs = data['Second Innings']['batters'][-1]['Total Runs']
        second_innings_overs = float(data['Second Innings']['batters'][-1]['Overs'])
        second_innings_wickets = data['Second Innings']['batters'][-1]['Wickets']
        second_innings_run_rate = calculate_run_rate(second_innings_overs, second_innings_runs)
        
        # Calculate the required run rate (assume 50-over match)
        required_run_rate = first_innings_run_rate
        
        # Get the adjective for chasing team performance
        chase_description = chasing_adjective(second_innings_run_rate, required_run_rate)
        
        # Add the second innings summary
        statement += (f" In response, {first_batting_team} came to defend the score of {first_innings_runs}, "
                      f"but {second_batting_team} produced a {chase_description} chase, scoring {second_innings_runs} runs in {second_innings_overs} overs, "
                      f"while taking {second_innings_wickets} wickets.")
    
    # Add the match conclusion
    statement += f" The match concluded with {data['match Info']['match result']}."
    return statement

In [6]:
def winning_Statement(data):
    
    first_batting_team = data["First Innings"]["batting Team"]
    second_batting_team = data["Second Innings"]["batting Team"]
    
    statement = (f"{first_batting_team} batted first and scored {data['First Innings']['batters'][-1]['Total Runs']} runs with "
                 f"{data['First Innings']['batters'][-1]['Wickets']} wickets in {data['First Innings']['batters'][-1]['Overs']} overs.")
    if len(data['Second Innings']['batters']) != 0:
        statement += (f"In response, {second_batting_team} scored {data['Second Innings']['batters'][-1]['Total Runs']} runs "
         f"in {data['Second Innings']['batters'][-1]['Overs']} overs with {data['Second Innings']['batters'][-1]['Wickets']} wickets.")
    statement += (f" The match concluded with {data['match Info']['match result']}")
    return statement

In [7]:
import math
def get_economy_adjective(economy_rate, required_run_rate=None):
    # If required run rate (RRR) is provided, adjust the adjective based on RRR comparison
    if required_run_rate:
        if required_run_rate < 4:
            # If RRR is less than 4, stricter economy expectations
            if economy_rate < 4:
                return "remarkable"
            elif 4 <= economy_rate < 6:
                return "disciplined but costly given the match context"
            elif 6 <= economy_rate < 8:
                return "costly, given the low required run rate"
            else:
                return "very expensive given the low required run rate"
        elif 4 <= required_run_rate < 6:
            # If RRR is between 4 and 6
            if economy_rate < 4:
                return "remarkable"
            elif 4 <= economy_rate < 6:
                return "disciplined"
            elif 6 <= economy_rate < 8:
                return "costly"
            else:
                return "expensive"
        elif 6 <= required_run_rate < 8:
            # If RRR is between 6 and 8, acceptable to bowl slightly higher
            if economy_rate < 4:
                return "remarkable given the required run rate"
            elif 4 <= economy_rate < 6:
                return "disciplined"
            elif 6 <= economy_rate < 8:
                return "competitive considering the required run rate"
            else:
                return "expensive given the match situation"
        else:
            # If RRR is greater than 8, it's a high chase, so lenient with higher economy
            if economy_rate < 4:
                return "remarkable considering the high required run rate"
            elif 4 <= economy_rate < 6:
                return "disciplined"
            elif 6 <= economy_rate < 8:
                return "effective in context of the high chase"
            else:
                return "expensive, but more acceptable given the high required run rate"
    else:
        # Default case when no required run rate is provided
        if economy_rate < 4:
            return "remarkable"
        elif 4 <= economy_rate < 6:
            return "disciplined"
        elif 6 <= economy_rate < 8:
            return "costly"
        else:
            return "expensive"

def firstInningBowlingSummaryPhrase(data, innings_v ,  run_rate = None):
    first_bowler = data[innings_v]["bowler"]    
    result = ""
    
    # Helper function to get economy adjective based on the rate
    
    
    for i in first_bowler:
        overs_bowled = float(i["overs"])
        economy_rate = float(i['Echonomy'])
        wickets_taken = int(i['Wickets'])

        total_allowed_overs = math.ceil(float(data['First Innings']["batters"][-1]["Overs"]) / 5)
        
        # Get the adjective for the economy rate
        economy_adjective = get_economy_adjective(economy_rate, run_rate)

        # Reporting when wickets were taken
        if wickets_taken != 0:
            if overs_bowled < total_allowed_overs:
                result += (i["bowler Name"] + " bowled " + i["overs"] + " overs, with a " + economy_adjective + " economy of " 
                           + i["Echonomy"] + ", took " + i["Wickets"] + " wickets, but did not complete their full bowling quota of overs. ")
            else:
                result += (i["bowler Name"] + " bowled their full bowling quota of overs with a " + economy_adjective + " economy of " 
                           + i["Echonomy"] + " and took " + i["Wickets"] + " wickets. ")
        # Reporting when no wickets were taken (bad performance)
        else:
            if overs_bowled < total_allowed_overs:
                result += (i["bowler Name"] + " had a tough day, bowled " + i["overs"] + " overs with a " + economy_adjective + " economy of " 
                           + i["Echonomy"] + ", failed to take any wickets, and didn’t bowl their full bowling quota of overs. ")
            else:
                result += (i["bowler Name"] + " had a tough day, bowled their full bowling quota of overs with a " + economy_adjective + " economy of " 
                           + i["Echonomy"] + " and failed to take any wickets. ")

    return result



In [8]:
def bowling_winnig_statement(data):
        # Merge of runs, wickets, and overs conditions
    statement = ""
    first_batting_team = data['First Innings']['batting Team']
    second_batting_team = data['Second Innings']['batting Team']
    temp1 = len(data['First Innings']['bowler'])-2
    statement += (f'{first_batting_team} uses {temp1} bowler in their bowling attack. ')
    if int(data['First Innings']['batters'][-1]['Total Runs']) >= 300:
        statement += (f"{second_batting_team}'s bowling struggled and allowed {first_batting_team} to post a high total "
                      f"of {data['First Innings']['batters'][-1]['Total Runs']} runs. ")
    elif int(data['First Innings']['batters'][-1]['Total Runs']) > 250:
        statement += (f"{second_batting_team} faced difficulty but managed to restrict {first_batting_team} "
                      f"to a decent score of {data['First Innings']['batters'][-1]['Total Runs']}. ")
    elif int(data['First Innings']['batters'][-1]['Total Runs']) > 200:
        statement += (f"{second_batting_team}'s bowling restricted {first_batting_team} to a manageable score of "
                      f"{data['First Innings']['batters'][-1]['Total Runs']}. ")
    else:
        statement += (f"{second_batting_team}'s bowling performed well, limiting {first_batting_team} "
                      f"to a low score of {data['First Innings']['batters'][-1]['Total Runs']}. ")

    # Now add wickets and overs information
    if int(data['First Innings']['batters'][-1]['Wickets']) == 10 and float(data['First Innings']['batters'][-1]['Overs']) < 50:
        statement += (f"They bowled out {first_batting_team} in just {data['First Innings']['batters'][-1]['Overs']} overs, "
                      f"taking all {data['First Innings']['batters'][-1]['Wickets']} wickets.")
    elif int(data['First Innings']['batters'][-1]['Wickets']) == 10:
        statement += (f"They managed to take all {data['First Innings']['batters'][-1]['Wickets']} wickets across the "
                      f"full {data['First Innings']['batters'][-1]['Overs']} overs.")
    elif int(data['First Innings']['batters'][-1]['Wickets']) >= 8 and float(data['First Innings']['batters'][-1]['Overs']) < 50:
        statement += (f"They put in a strong bowling performance, taking {data['First Innings']['batters'][-1]['Wickets']} wickets "
                      f"in {data['First Innings']['batters'][-1]['Overs']} overs.")
    elif int(data['First Innings']['batters'][-1]['Wickets']) >= 8:
        statement += (f"They took {data['First Innings']['batters'][-1]['Wickets']} wickets, but {first_batting_team} survived "
                      f"the full {data['First Innings']['batters'][-1]['Overs']} overs.")
    elif int(data['First Innings']['batters'][-1]['Wickets']) >= 5:
        statement += (f"{second_batting_team} managed to take {data['First Innings']['batters'][-1]['Wickets']} wickets over "
                      f"{data['First Innings']['batters'][-1]['Overs']} overs, but {first_batting_team}'s batsmen put up a fight.")
    elif int(data['First Innings']['batters'][-1]['Wickets']) >= 2:
        statement += (f"They took {data['First Innings']['batters'][-1]['Wickets']} wickets in {data['First Innings']['batters'][-1]['Overs']} overs, "
                      f"but {first_batting_team} still managed to put up a respectable total.")
    else:
        statement += (f"{first_batting_team} faced minimal difficulty from {second_batting_team}'s bowling, losing only "
                      f"{data['First Innings']['batters'][-1]['Wickets']} wicket(s) in {data['First Innings']['batters'][-1]['Overs']} overs.")
        
    statement += (f" In response {second_batting_team} come to chase the score and {data['match Info']['match result']}. ")
    
    # now in responce team-b chase
    if len(data['Second Innings']['batters']) > 0:
        statement += (f" {first_batting_team} consede {data['Second Innings']['batters'][-1]['Total Runs']} in {data['Second Innings']['batters'][-1]['Overs']} and took {data['Second Innings']['batters'][-1]['Wickets']} Wickets")
        
    
    return statement

In [9]:
def firstInningBowlingPhrase(data, txt):
    array = data[txt]["bowler"]
    
    result = ""
    for i in array:
        if float(i['Echonomy']) < 7 and int(i['Wickets']) != 0:
            result += i["bowler Name"] + " did good economical bowling with " + i["Echonomy"] + " and took " + i["Wickets"] + " wickets."
        elif float(i['Echonomy']) < 7:
            result += i["bowler Name"] + " creatend pressure and didn't leak runs with the echonmy of " + i["Echonomy"] + ". "
        if float(i['Echonomy']) >= 7 and int(i['Wickets']) !=0:
            result += i["bowler Name"] + " was expesive with the economical of " + i["Echonomy"] + " but took " + i["Wickets"] + " wickets. "
    return result

In [10]:
def firstInningsBattingPhrase(data, txt):
    return_statement = ""
    firstInningsBatting = data[txt]["batters"]
    
    for index, i in enumerate(firstInningsBatting):
        if "Extras" not in i and "Total Runs" not in i:
            temp = evaluate_batter_performance(i['runs'], i['stric rate'])
            if temp != "Don't mention":
                return_statement += ( "" + i["batter name"] + " comes at " + str(index + 1) + " position and give a " + temp + 
                                    "and scores " + i["runs"] + " with a strick rate of " + i["stric rate"] + ". ")            
                
    return return_statement


In [11]:
def evaluate_batter_performance(runs, strike_rate):
    runs = int(runs)
    strike_rate = float(strike_rate)

    # Evaluating based on runs and strike rate
    if runs < 10:
        if strike_rate < 50:
            return "Struggled to get going"
        elif 50 <= strike_rate < 80:
            return "Slow start, couldn't capitalize"
        elif 80 <= strike_rate < 120:
            return "Aggressive but fell early"
        else:  # strike_rate >= 120
            return "Brief cameo, fell early"

    if 10 <= runs < 20:
        if strike_rate < 50:
            return "Couldn't accelerate"
        elif 50 <= strike_rate < 80:
            return "Started slow, but didn't convert"
        elif 80 <= strike_rate < 120:
            return "Under pressure, quick runs"
        else:  # strike_rate >= 120
            return "Aggressive intent, quick but short-lived"

    if 20 <= runs < 50:
        if strike_rate < 50:
            return "Struggled to rotate strike"
        elif 50 <= strike_rate < 80:
            return "Slow start, steady innings"
        elif 80 <= strike_rate < 120:
            return "Controlled aggression, built momentum"
        else:  # strike_rate >= 120
            return "Blistering start, impactful knock"

    if runs >= 50:
        if strike_rate < 50:
            return "Played a gritty innings"
        elif 50 <= strike_rate < 80:
            return "Anchor role, crucial stability"
        elif 80 <= strike_rate < 120:
            return "Solid performance, balanced aggression"
        else:  # strike_rate >= 120
            return "Explosive innings, dominated the bowlers"

    # Fallback in case none of the conditions are met
    return "Performance could not be evaluated"


In [12]:
def batting_summary_BattingPhrase(data, txt):
    return_statement = ""
    firstInningsBatting = data[txt]["batters"]
    
    for index, i in enumerate(firstInningsBatting):
        if "Extras" not in i and "Total Runs" not in i:
            temp = evaluate_batter_performance(i['runs'], i['stric rate'])
            if temp != "Don't mention":
                return_statement += ( "" + i["batter name"] + " comes at " + str(index + 1) + " position and give a " + temp + 
                                    "and scores " + i["runs"] + " with a strick rate of " + i["stric rate"] + ". ")            
                
    return return_statement

In [13]:
# overall summary
def overall_summary(i):
    parts = i['match Info']['match'].split(',')

    # Extract "England vs New Zealand"
    teams = parts[0].strip()
    teamA = i["First Innings"]["batting Team"]
    teamB = i["Second Innings"]["batting Team"]
    
    firstInnings = i["First Innings"]
    secondInnings = i["Second Innings"]
    match_info = (''.join(i["match Info"]['match'].split(',')[1:]))
    # print(i["match Info"]['match'].split(',')[-1])
    paragraph = (
        f"The match between {teamA} vs {teamB} was the {match_info}. "
        f"In the {i['match Info']['Series']}, this match took place at the {i['match Info']['venue']} on {i['match Info']['time']}, {i['match Info']['date']}. "
        f"The exciting encounter concluded with {i['match Info']['match result'].strip()}. {winning_Statement(i)}."
        f"{firstInningsBattingPhrase(i, 'First Innings')} In bowling by {teamB} ,{firstInningBowlingPhrase(i, 'First Innings')}"
        f" {firstInningsBattingPhrase(i, 'Second Innings')}  In bowling by {teamA} {firstInningBowlingPhrase(i, 'Second Innings')} "
    )
    return paragraph

In [14]:
#bowling summary
def bowling_summary(i):
    teamA = i["First Innings"]["batting Team"]
    teamB = i["Second Innings"]["batting Team"]
    match_info = (''.join(i["match Info"]['match'].split(',')[1:]))

    teamA_run_rate = calculate_run_rate(i['First Innings']['batters'][-1]['Overs'], i['First Innings']['batters'][-1]['Total Runs'])
    paragraph = (
        f"The match between {teamA} vs {teamB} was the{match_info}. In the {i['match Info']['Series']},"
        f" this match took place at the {i['match Info']['venue']} on {i['match Info']['time']},"
        f" {i['match Info']['date']}. {generate_bowling_match_summary(i)}."
        f" In {teamB}'s bowling attack,{firstInningBowlingSummaryPhrase(i, 'First Innings')}")

    if len(i['Second Innings']['batters']) != 0:
        paragraph += (
            f"In response {teamA}'s bowler come to defend score of {i['First Innings']['batters'][-1]['Total Runs']}."
            f"In {teamA}'s bowling attack,{firstInningBowlingSummaryPhrase(i, 'Second Innings', teamA_run_rate)} ")
    return paragraph

The match between ENGU19 vs PAKU19 was the  Warm-up Matches ICC Under 19 World Cup Warm up Matches 2024 . In the ICC Cricket World Cup Qualifiers 2023, this match took place at the  Queens Sports Club, Bulawayo  on 12:30 PM, Wednesday, June 21, 2023. The exciting encounter concluded with Scotland won by 1 wkt.In PAKU19's bowling attack,Boult bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 1 wickets. Matt Henry bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 3 wickets. Santner bowled their full quota of 10 overs, did good economical bowling with an economy of 3.70 and took 2 wickets. Rachin Ravindra was expensive with an economy of 7.60, bowled their full 10 overs, and took 1 wickets. Glenn Phillips bowled 3 overs, did good economical bowling with an economy of 5.70, took 2 wickets, but did not complete their full quota of 10 overs. Neesham had a tough day, bowled 7 overs with an expensiv

The match between ENGU19 vs PAKU19 was the  Warm-up Matches ICC Under 19 World Cup Warm up Matches 2024 . In the ICC Cricket World Cup Qualifiers 2023, this match took place at the  Bulawayo Athletic Club, Bulawayo  on 12:30 PM, Wednesday, June 21, 2023. The exciting encounter concluded with Oman won by 5 wkts.In PAKU19's bowling attack,Boult bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 1 wickets. Matt Henry bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 3 wickets. Santner bowled their full quota of 10 overs, did good economical bowling with an economy of 3.70 and took 2 wickets. Rachin Ravindra was expensive with an economy of 7.60, bowled their full 10 overs, and took 1 wickets. Glenn Phillips bowled 3 overs, did good economical bowling with an economy of 5.70, took 2 wickets, but did not complete their full quota of 10 overs. Neesham had a tough day, bowled 7 overs with an expensi

The match between ENGU19 vs PAKU19 was the  Warm-up Matches ICC Under 19 World Cup Warm up Matches 2024 . In the ICC Cricket World Cup Qualifiers 2023, this match took place at the  Bulawayo Athletic Club, Bulawayo  on 12:30 PM, Wednesday, June 21, 2023. The exciting encounter concluded with Oman won by 5 wkts.In PAKU19's bowling attack,Boult bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 1 wickets. Matt Henry bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 3 wickets. Santner bowled their full quota of 10 overs, did good economical bowling with an economy of 3.70 and took 2 wickets. Rachin Ravindra was expensive with an economy of 7.60, bowled their full 10 overs, and took 1 wickets. Glenn Phillips bowled 3 overs, did good economical bowling with an economy of 5.70, took 2 wickets, but did not complete their full quota of 10 overs. Neesham had a tough day, bowled 7 overs with an expensi

The match between ENGU19 vs PAKU19 was the  Warm-up Matches ICC Under 19 World Cup Warm up Matches 2024 . In the ICC Cricket World Cup Qualifiers 2023, this match took place at the  Takashinga Sports Club, Harare  on 12:30 PM, Thursday, June 22, 2023. The exciting encounter concluded with Netherlands won by 5 wkts.In PAKU19's bowling attack,Boult bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 1 wickets. Matt Henry bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 3 wickets. Santner bowled their full quota of 10 overs, did good economical bowling with an economy of 3.70 and took 2 wickets. Rachin Ravindra was expensive with an economy of 7.60, bowled their full 10 overs, and took 1 wickets. Glenn Phillips bowled 3 overs, did good economical bowling with an economy of 5.70, took 2 wickets, but did not complete their full quota of 10 overs. Neesham had a tough day, bowled 7 overs with an exp

The match between ENGU19 vs PAKU19 was the  Warm-up Matches ICC Under 19 World Cup Warm up Matches 2024 . In the ICC Cricket World Cup Qualifiers 2023, this match took place at the  Takashinga Sports Club, Harare  on 12:30 PM, Thursday, June 22, 2023. The exciting encounter concluded with Netherlands won by 5 wkts.In PAKU19's bowling attack,Boult bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 1 wickets. Matt Henry bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 3 wickets. Santner bowled their full quota of 10 overs, did good economical bowling with an economy of 3.70 and took 2 wickets. Rachin Ravindra was expensive with an economy of 7.60, bowled their full 10 overs, and took 1 wickets. Glenn Phillips bowled 3 overs, did good economical bowling with an economy of 5.70, took 2 wickets, but did not complete their full quota of 10 overs. Neesham had a tough day, bowled 7 overs with an exp

...

The match between ENGU19 vs PAKU19 was the  Warm-up Matches ICC Under 19 World Cup Warm up Matches 2024 . In the ICC Cricket World Cup Qualifiers 2023, this match took place at the  Harare Sports Club, Harare  on 12:30 PM, Friday, July 07, 2023. The exciting encounter concluded with Sri Lanka won by 8 wkts.In PAKU19's bowling attack,Boult bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 1 wickets. Matt Henry bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 3 wickets. Santner bowled their full quota of 10 overs, did good economical bowling with an economy of 3.70 and took 2 wickets. Rachin Ravindra was expensive with an economy of 7.60, bowled their full 10 overs, and took 1 wickets. Glenn Phillips bowled 3 overs, did good economical bowling with an economy of 5.70, took 2 wickets, but did not complete their full quota of 10 overs. Neesham had a tough day, bowled 7 overs with an expensive e

The match between ENGU19 vs PAKU19 was the  Warm-up Matches ICC Under 19 World Cup Warm up Matches 2024 . In the ICC Cricket World Cup Qualifiers 2023, this match took place at the  Harare Sports Club, Harare  on 12:30 PM, Sunday, July 09, 2023. The exciting encounter concluded with Sri Lanka won by 128 runs.In PAKU19's bowling attack,Boult bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 1 wickets. Matt Henry bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 3 wickets. Santner bowled their full quota of 10 overs, did good economical bowling with an economy of 3.70 and took 2 wickets. Rachin Ravindra was expensive with an economy of 7.60, bowled their full 10 overs, and took 1 wickets. Glenn Phillips bowled 3 overs, did good economical bowling with an economy of 5.70, took 2 wickets, but did not complete their full quota of 10 overs. Neesham had a tough day, bowled 7 overs with an expensive

The match between ENGU19 vs PAKU19 was the  Warm-up Matches ICC Under 19 World Cup Warm up Matches 2024 . In the ICC Cricket World Cup Qualifiers 2023, this match took place at the  Harare Sports Club, Harare  on 12:30 PM, Sunday, July 09, 2023. The exciting encounter concluded with Sri Lanka won by 128 runs.In PAKU19's bowling attack,Boult bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 1 wickets. Matt Henry bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 3 wickets. Santner bowled their full quota of 10 overs, did good economical bowling with an economy of 3.70 and took 2 wickets. Rachin Ravindra was expensive with an economy of 7.60, bowled their full 10 overs, and took 1 wickets. Glenn Phillips bowled 3 overs, did good economical bowling with an economy of 5.70, took 2 wickets, but did not complete their full quota of 10 overs. Neesham had a tough day, bowled 7 overs with an expensive

The match between ENGU19 vs PAKU19 was the  Warm-up Matches ICC Under 19 World Cup Warm up Matches 2024 . In the ICC Cricket World Cup Qualifier Play-off 2023, this match took place at the  Wanderers Cricket Ground, Windhoek  on 01:00 PM, Sunday, March 26, 2023. The exciting encounter concluded with United States won by 80 runs.In PAKU19's bowling attack,Boult bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 1 wickets. Matt Henry bowled their full quota of 10 overs, did good economical bowling with an economy of 4.80 and took 3 wickets. Santner bowled their full quota of 10 overs, did good economical bowling with an economy of 3.70 and took 2 wickets. Rachin Ravindra was expensive with an economy of 7.60, bowled their full 10 overs, and took 1 wickets. Glenn Phillips bowled 3 overs, did good economical bowling with an economy of 5.70, took 2 wickets, but did not complete their full quota of 10 overs. Neesham had a tough day, bowled 7 ove

KeyboardInterrupt: 

In [15]:
# batting summary

def batting_summary(i):
    teamA = i["First Innings"]["batting Team"]
    teamB = i["Second Innings"]["batting Team"]
    match_info = (''.join(i["match Info"]['match'].split(',')[1:]))

    teamA_run_rate = calculate_run_rate(i['First Innings']['batters'][-1]['Overs'], i['First Innings']['batters'][-1]['Total Runs'])
    paragraph = (
            f"The match between {teamA} vs {teamB} was the{match_info}. In the {i['match Info']['Series']},"
            f" this match took place at the {i['match Info']['venue']} on {i['match Info']['time']},"
            f" {i['match Info']['date']}. {generate_batting_match_summary(i)}."
            f" In {teamA}'s batting,{batting_summary_BattingPhrase(i,'First Innings')}"
            )
    if len(i['Second Innings']['batters']) != 0:
        paragraph += (
            f"In response {teamB}'s batters come to achive the score of {i['First Innings']['batters'][-1]['Total Runs']}."
            f"{teamB}'s batters,{batting_summary_BattingPhrase(i,'Second Innings')} ")
    return paragraph

In [20]:
import csv
headers = ['Match Info', 'Overall Summary', 'Bowling Summary', 'Batting Summary']

# Assuming 'data' is a list or iterable of items to process
for i in data:
    # Open the CSV file in append mode ('a') instead of write mode ('w')
    with open('cricket_summary.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Create a list containing the summary details
        lst = [str(i), overall_summary(i), bowling_summary(i), batting_summary(i)]
        
        # Write the list as a new row in the CSV file
        writer.writerow(lst)
        
        # Print the length of each element in the list
        for l in lst:
            print(len(l), end=',')
        print('\n')  # Print a new line after each row


4797,2765,2304,2386,

6394,3992,2632,3348,

5350,3343,2394,2733,

5408,3483,2250,2927,

5225,3292,2271,2626,

5946,3800,2498,3188,

6071,3857,2303,3348,

5472,3322,2633,2683,

4768,2676,2392,2351,

5706,3755,2389,3100,

5166,3169,2316,2621,

4946,3029,2161,2600,

6154,3899,2429,3314,

5484,3444,2386,2919,

5826,3904,2124,3306,

5625,3655,2347,2981,

5169,3190,2428,2509,

6047,3883,2444,3333,

5578,3562,2460,2927,

5562,3539,2312,3083,

5391,3578,2050,2977,

4550,2867,2284,2187,

5531,3493,2415,2875,

5907,3821,2378,3212,

5027,2815,2379,2430,

6136,3984,2279,3406,

6190,3832,2583,3388,

6158,3917,2271,3240,

5661,3629,2193,3114,

5226,3294,2363,2671,

5073,3236,2249,2595,

5290,3296,2469,2777,

5660,3594,2319,3044,

5224,3179,2673,2664,

4415,2341,2613,2085,

5989,3863,2169,3279,

5247,3324,2211,2813,

5745,3709,2175,3144,

5227,3181,2460,2596,

5937,3790,2217,3278,

5394,3311,2361,2876,

5603,3401,2454,2879,

4828,2821,2492,2449,

5846,3805,2037,3283,

5742,3408,2894,2796,

5172,3219,